In [31]:
import asyncio
from dotenv import load_dotenv
import aiohttp

load_dotenv()
import nest_asyncio

nest_asyncio.apply()
import os
from anthropic import AsyncAnthropic
from xml.sax.saxutils import escape
from suno import SongsGen
from concurrent.futures import ThreadPoolExecutor

from io import BytesIO
import base64

In [32]:

async def async_generate_music(lyrics):
    print("Generating music")
    GenerateSong = SongsGen(os.environ.get("SUNO_COOKIE"))
    print(GenerateSong.get_limit_left())
    loop = asyncio.get_running_loop()

    result = None
    # Use a ThreadPoolExecutor to run synchronous functions in threads
    with ThreadPoolExecutor() as pool:
        result = await loop.run_in_executor(
            pool,
            lambda: GenerateSong.get_songs(lyrics, is_custom = True, title = "custom", tags = "slow, catchy, maritime, pirate"),
        )

    if not result:
        return None

    link = result["song_url"]
    print("Link: ", link)
    
    response = GenerateSong.session.get(link, stream = True)
    print(response.status_code)
    audio = response.iter_content()
    data = b''
    for chunk in audio:
        if chunk:
            # translates binary to base64 string
            b64 = base64.b64encode(chunk)
            # decodes the b.64 binary obj into a string
            utf = b64.decode('utf-8')
            # creates a dict obj that stores the event as an audio chunk and sets the audio data to utf format
            obj = {
                "event": "audio",
                "audio_data": utf
            }
            # waits for broadcasting to run
            data += chunk
            
    return data
             

In [33]:
data =  """
Verse 1:
When yer sailin' on the seas (yo-ho!)
And you've got some tasty treats (arr!)
Don't forget to share with mates
That's what every pirate states! (ahoy!)
"""   

audio = asyncio.run(async_generate_music(data))


Generating music
89


{
    'gpt_description_prompt': '',
    'mv': 'chirp-v3-0',
    'prompt': "\nVerse 1:\nWhen yer sailin' on the seas (yo-ho!)\nAnd you've got some tasty treats (arr!)\nDon't 
forget to share with mates\nThat's what every pirate states! (ahoy!)\n",
    'make_instrumental': False,
    'title': 'custom',
    'tags': 'slow, catchy, maritime, pirate'
}

Waiting for results...

.

.

.

Link:  https://audiopipe.suno.ai/?item_id=0276d537-86ff-401f-a2c0-6a542b63b9a2
200


In [34]:
# binary is a base64 encoded string, save it to a file
with open("custom_song.mp3", "wb") as f:
    f.write(audio)

In [35]:
from openai import AsyncOpenAI
import tempfile
client = AsyncOpenAI()
async def transcribe_audio(audio):
    with tempfile.NamedTemporaryFile(
            suffix=".mp3", mode="wb", delete=True
        ) as temp_audio_file:
        temp_audio_file.write(audio)
        temp_audio_file.flush()
        temp_audio_file.seek(0)

        with open(temp_audio_file.name, "rb") as audio_file:
            transcript_obj = await client.audio.transcriptions.create(
                model="whisper-1", file=audio_file, response_format="verbose_json", timestamp_granularities=['word']
            )
    return transcript_obj.words

transcript = asyncio.run(transcribe_audio(audio))

In [39]:
transcript

[{'word': 'When', 'start': 12.579999923706055, 'end': 13.720000267028809},
 {'word': "you're", 'start': 13.720000267028809, 'end': 14.180000305175781},
 {'word': 'sailing', 'start': 14.180000305175781, 'end': 14.760000228881836},
 {'word': 'on', 'start': 14.760000228881836, 'end': 15.479999542236328},
 {'word': 'the', 'start': 15.479999542236328, 'end': 16.059999465942383},
 {'word': 'sea', 'start': 16.059999465942383, 'end': 16.200000762939453},
 {'word': 'and', 'start': 17.440000534057617, 'end': 17.440000534057617},
 {'word': 'you', 'start': 17.440000534057617, 'end': 17.940000534057617},
 {'word': 'got', 'start': 17.940000534057617, 'end': 18.260000228881836},
 {'word': 'some', 'start': 18.260000228881836, 'end': 19.059999465942383},
 {'word': 'tasty', 'start': 19.059999465942383, 'end': 19.139999389648438},
 {'word': 'treats', 'start': 19.139999389648438, 'end': 19.600000381469727},
 {'word': "don't", 'start': 20.440000534057617, 'end': 20.84000015258789},
 {'word': 'forget', 'sta